<a href="https://colab.research.google.com/github/Vika0408/Mental-Health/blob/main/helpingminds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kagglehub

In [ ]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("zaber666/meld-dataset")

print("Path to dataset files:", path)


Path to dataset files: /root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
DATASET_PATH = "/root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1"


In [ ]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1/MELD-RAW/MELD.Raw/train/train_sent_emo.csv')

In [ ]:
# Extract Text & Labels
texts = df["Utterance"].astype(str).tolist()
sentiments = df["Sentiment"].tolist()  # Positive, Negative, Neutral
emotions = df["Emotion"].tolist()  # Excited, Sad, Angry, etc.


In [ ]:
# Tokenize Text
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

In [ ]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

In [ ]:
# Convert Sentiments & Emotions to Numeric Labels
sentiment_encoder = LabelEncoder()
sentiment_labels = sentiment_encoder.fit_transform(sentiments)

In [ ]:
emotion_encoder = LabelEncoder()
emotion_labels = emotion_encoder.fit_transform(emotions)

In [ ]:
# Split into Train & Test Sets
X_train, X_test, y_train_sentiment, y_test_sentiment, y_train_emotion, y_test_emotion = train_test_split(
    padded_sequences, sentiment_labels, emotion_labels, test_size=0.2, random_state=42
)

In [ ]:
# Convert labels to NumPy arrays to ensure uniform shape
y_train_sentiment = np.array(y_train_sentiment)
y_test_sentiment = np.array(y_test_sentiment)
y_train_emotion = np.array(y_train_emotion)
y_test_emotion = np.array(y_test_emotion)

In [ ]:
# Save Preprocessed Data
np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("y_train_sentiment.npy", y_train_sentiment)
np.save("y_test_sentiment.npy", y_test_sentiment)
np.save("y_train_emotion.npy", y_train_emotion)
np.save("y_test_emotion.npy", y_test_emotion)

In [ ]:
# Save Tokenizer & Label Mappings
with open("tokenizer.pkl", "wb") as handle:
    pickle.dump(tokenizer, handle)

In [ ]:
with open("sentiment_labels.pkl", "wb") as handle:
    pickle.dump(sentiment_encoder.classes_, handle)

In [ ]:
with open("emotion_labels.pkl", "wb") as handle:
    pickle.dump(emotion_encoder.classes_, handle)

In [ ]:
print("✅ Data preprocessing complete! Files saved.")

✅ Data preprocessing complete! Files saved.


In [ ]:
import numpy as np

# Reload preprocessed data
X_train, X_test, y_train_sentiment, y_test_sentiment, y_train_emotion, y_test_emotion = train_test_split(
    padded_sequences, sentiment_labels, emotion_labels, test_size=0.2, random_state=42
)

# Ensure all arrays have the correct shape before saving
print("Shapes:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train_sentiment: {y_train_sentiment.shape}, y_test_sentiment: {y_test_sentiment.shape}")
print(f"y_train_emotion: {y_train_emotion.shape}, y_test_emotion: {y_test_emotion.shape}")

# Save data separately in a compressed file
np.savez_compressed("train_test_data.npz",
                    X_train=X_train, X_test=X_test,
                    y_train_sentiment=y_train_sentiment, y_test_sentiment=y_test_sentiment,
                    y_train_emotion=y_train_emotion, y_test_emotion=y_test_emotion)

print("✅ Data successfully saved!")


Shapes:
X_train: (7991, 100), X_test: (1998, 100)
y_train_sentiment: (7991,), y_test_sentiment: (1998,)
y_train_emotion: (7991,), y_test_emotion: (1998,)
✅ Data successfully saved!


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import numpy as np
import pickle

In [ ]:
import os

file_path = "train_test_data.npz"
if os.path.exists(file_path):
    print(f"File exists. Size: {os.path.getsize(file_path)} bytes")
else:
    print("File does not exist.")


File does not exist.


In [ ]:
import os

file_path = "train_test_data.npz"
if os.path.exists(file_path):
    os.remove(file_path)
    print("✅ Deleted empty file. Ready to save again.")


✅ Deleted empty file. Ready to save again.


In [ ]:
import numpy as np

# Convert all lists to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train_sentiment = np.array(y_train_sentiment)
y_test_sentiment = np.array(y_test_sentiment)
y_train_emotion = np.array(y_train_emotion)
y_test_emotion = np.array(y_test_emotion)

# Ensure all arrays have the same length in the first dimension
print(f"Shapes:\nX_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train_sentiment: {y_train_sentiment.shape}, y_test_sentiment: {y_test_sentiment.shape}")
print(f"y_train_emotion: {y_train_emotion.shape}, y_test_emotion: {y_test_emotion.shape}")

# Save as a dictionary instead
data_dict = {
    "X_train": X_train,
    "X_test": X_test,
    "y_train_sentiment": y_train_sentiment,
    "y_test_sentiment": y_test_sentiment,
    "y_train_emotion": y_train_emotion,
    "y_test_emotion": y_test_emotion
}

# Save using np.savez (handles multiple arrays properly)
np.savez("train_test_data.npz", **data_dict)
print("✅ Data successfully saved as .npz!")


Shapes:
X_train: (3600, 100), X_test: (1000, 100)
y_train_sentiment: (3600,), y_test_sentiment: (1000,)
y_train_emotion: (3600,), y_test_emotion: (1000,)
✅ Data successfully saved as .npz!


In [ ]:
loaded_data = np.load("train_test_data.npz")

X_train = loaded_data["X_train"]
X_test = loaded_data["X_test"]
y_train_sentiment = loaded_data["y_train_sentiment"]
y_test_sentiment = loaded_data["y_test_sentiment"]
y_train_emotion = loaded_data["y_train_emotion"]
y_test_emotion = loaded_data["y_test_emotion"]

print("✅ Data successfully loaded!")


✅ Data successfully loaded!


In [ ]:
import os

file_path = "train_test_data.npz"
if os.path.exists(file_path):
    print(f"File exists. Size: {os.path.getsize(file_path)} bytes")
else:
    print("File does not exist.")

File exists. Size: 3718364 bytes


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (Embedding, LSTM, Dense, Bidirectional, Dropout, Input,
                                     MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D, Masking)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import AdamW

# Define model parameters
VOCAB_SIZE = 10000
EMBEDDING_DIM = 128
LSTM_UNITS = 512  # Increased for better learning
MAX_SEQUENCE_LENGTH = 100
DROPOUT_RATE = 0.3  # Reduced to prevent excessive dropout

# Input layer
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,))

# Embedding layer with masking
x = Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(inputs)

# Bidirectional LSTM with dropout
x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True, dropout=DROPOUT_RATE))(x)

# Multi-Head Self-Attention (fixes previous Attention layer issues)
attention_output = MultiHeadAttention(num_heads=4, key_dim=LSTM_UNITS)(x, x)
x = LayerNormalization()(x + attention_output)

# Global Pooling
x = GlobalAveragePooling1D()(x)

# Sentiment output
sentiment_output = Dense(3, activation="softmax", name="sentiment_output")(x)

# Emotion output
emotion_output = Dense(7, activation="softmax", name="emotion_output")(x)

# Build the model
model = Model(inputs, outputs=[sentiment_output, emotion_output])

# Compile the model with AdamW optimizer and lower learning rate
model.compile(
    optimizer=AdamW(learning_rate=0.0003, weight_decay=1e-5),
    loss={"sentiment_output": "sparse_categorical_crossentropy", "emotion_output": "sparse_categorical_crossentropy"},
    metrics={"sentiment_output": "accuracy", "emotion_output": "accuracy"},
)


In [ ]:
# Model Summary
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 100, 128)       │      1,280,000 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_4 (NotEqual)    │ (None, 100)            │              0 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_4           │ (None, 100, 1024)      │      2,625,536 │ embedding_4[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 100, 1024)      │      8,395,776 │ bidirectional_4[0][0], │
│ (MultiHeadAttention)      │                        │                │ bidirectional_4[0][0], │
│                           │                        │                │ not_equal_4[0][0],     │
│                           │                        │                │ not_equal_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_4 (Add)               │ (None, 100, 1024)      │              0 │ bidirectional_4[0][0], │
│                           │                        │                │ multi_head_attention[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 100, 1024)      │          2,048 │ add_4[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 1024)           │              0 │ layer_normalization[0… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sentiment_output (Dense)  │ (None, 3)              │          3,075 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ emotion_output (Dense)    │ (None, 7)              │          7,175 │ global_average_poolin… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 12,313,610 (46.97 MB)

 Trainable params: 12,313,610 (46.97 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Dummy Data Loading (Replace with actual dataset)
# Assuming X, y_sentiment, and y_emotion are loaded and preprocessed
X = np.random.randint(1, VOCAB_SIZE, (5000, MAX_SEQUENCE_LENGTH))
y_sentiment = np.random.randint(0, 3, 5000)
y_emotion = np.random.randint(0, 7, 5000)


In [ ]:
# Splitting dataset properly
X_train, X_val, y_train_sentiment, y_val_sentiment, y_train_emotion, y_val_emotion = train_test_split(
    X, y_sentiment, y_emotion, test_size=0.2, random_state=42
)

In [ ]:
X_train, X_val, y_train_sentiment, y_val_sentiment, y_train_emotion, y_val_emotion = train_test_split(
    X_train, y_train_sentiment, y_train_emotion, test_size=0.1, random_state=42
)

In [ ]:
# Ensure labels are correctly formatted as NumPy arrays
y_train_sentiment = np.array(y_train_sentiment, dtype=np.int32)
y_test_sentiment = np.array(y_test_sentiment, dtype=np.int32)
y_val_sentiment = np.array(y_val_sentiment, dtype=np.int32)

y_train_emotion = np.array(y_train_emotion, dtype=np.int32)
y_test_emotion = np.array(y_test_emotion, dtype=np.int32)
y_val_emotion = np.array(y_val_emotion, dtype=np.int32)

# Debugging: Check the shapes
print("Shapes:")
print("X_train:", X_train.shape)
print("y_train_sentiment:", y_train_sentiment.shape, "Unique:", np.unique(y_train_sentiment))
print("y_train_emotion:", y_train_emotion.shape, "Unique:", np.unique(y_train_emotion))


Shapes:
X_train: (3600, 100)
y_train_sentiment: (3600,) Unique: [0 1 2]
y_train_emotion: (3600,) Unique: [0 1 2 3 4 5 6]


In [ ]:
print("Expected Input Shape:", model.input_shape)
print("Actual X_train Shape:", X_train.shape)


Expected Input Shape: (None, 100)
Actual X_train Shape: (3600, 100)


In [ ]:
# Train the model
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train,
    {"sentiment": y_train_sentiment, "emotion": y_train_emotion},
    epochs=30,
    batch_size=64,
    validation_data=(X_test, {"sentiment": y_test_sentiment, "emotion": y_test_emotion}),
    callbacks=[early_stopping]
)


Epoch 1/30


ValueError: Attr 'Toutput_types' of 'OptionalFromValue' Op passed list of length 0 less than minimum 1.